In [1]:
from sequence_example_lib import *
import os
import tensorflow as tf
import numpy as np
import time

sequence_example_file = ('/Users/mac/Desktop/Brain/Simplified_Magenta/Wikifonia_basic_rnn_sequence_examples/training_melodies.tfrecord')
sequence_example_file_paths = tf.gfile.Glob(
    os.path.expanduser(sequence_example_file))

start_time = time.time()
inputs, labels, lengths = get_numpy_from_tf_sequence_example( input_size=38,
                                    sequence_example_file_paths = sequence_example_file_paths,
                                    shuffle = False)
print('Time:',time.time() - start_time)
print('inputs shape',inputs.shape)
print('inputs type',type(inputs))
print('Finished! And then use ipdb for debug and watch the variables')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Counting records in /Users/mac/Desktop/Brain/Simplified_Magenta/Wikifonia_basic_rnn_sequence_examples/training_melodies.tfrecord.
INFO:tensorflow:Total records: 6668
INFO:tensorflow:[<tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:0' shape=(?, 38) dtype=float32>, <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, <tf.Tensor 'strided_slice:0' shape=() dtype=int32>]
inputs (6668, 511, 38) <class 'numpy.ndarray'> [ 0  0  0  0  0  0  0  0 18  0]
labels (6668, 511) <class 'numpy.ndarray'> [ 0  0  0  0  0  0  0 18  0  0]
Time: 2.7724790573120117
inputs shape (6668, 511, 38)
inputs type <class 'numpy.ndarray'>
Finished! And then use ipdb for debug and watch the variables


In [2]:
def to_events(x):
    return np.argmax(x, axis=1)

In [3]:
input_events = []
for i in inputs:
    input_events.append(to_events(i))

In [4]:
def to_real_length(x):
    while(x[len(x)-1]==0):
        x.pop()

In [5]:
real_inputs = []
for i in input_events:
    d = list(i)
    to_real_length(d)
    real_inputs.append(d)

In [6]:
real_length = []
for i in real_inputs:
    real_length.append(len(i))
real_length = np.array(real_length)

In [7]:
print(real_length.min())
print(real_length.max())
print(real_length.mean())

49
511
359.3848230353929


In [8]:
file = open("len.txt","w")
for i in real_length:
    file.write("%d\n"%i)
file.close()

In [9]:
def zero_rate(x):
    x = np.array(x)
    non_zero_num = len(list(x.nonzero()[0]))
    return 1-non_zero_num*1.0/len(x)

In [10]:
zero_rates = []
for i in real_inputs:
    zero_rates.append(zero_rate(i))
zero_rates = np.array(zero_rates)

In [11]:
print(zero_rates.min())
print(zero_rates.max())
print(zero_rates.mean())

0.03522504892367906
0.9348441926345609
0.7312030925153147


In [12]:
for i in range(3):
    print(real_inputs[i])

[0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 21, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 21, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 23, 0, 0, 0, 0, 0, 0, 0, 23, 0, 0, 0, 23, 0, 0, 0, 21, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 16, 0, 0, 0, 14, 0, 0, 0, 16, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 1, 0, 0, 0, 18, 0, 0, 0, 21, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 14, 0, 0, 0, 23, 0, 19, 0, 21, 0, 0, 0, 23, 0, 0, 0, 23, 0, 0, 0, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 21, 0, 18, 0, 19, 0, 0, 0, 21, 0, 0, 0, 21, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 14, 0, 0, 0, 23, 0, 19, 0, 21, 0, 0, 0, 23, 0, 0, 0, 23, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 0, 25, 0, 26, 0, 0, 0, 25, 0, 26, 0, 25, 0, 0, 0, 23, 0, 0, 0, 16, 0, 0, 0, 1, 0, 0, 0, 18, 0, 0, 0, 21, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 26, 0,

In [13]:
from my_to_midi import *

Using TensorFlow backend.


In [14]:
for i in range(1,11):
    a = real_inputs[10]
    events_to_midi(a, 'test_midi', 'input%d'%i)

INFO:tensorflow:Wrote input1.mid to test_midi
INFO:tensorflow:Wrote input2.mid to test_midi
INFO:tensorflow:Wrote input3.mid to test_midi
INFO:tensorflow:Wrote input4.mid to test_midi
INFO:tensorflow:Wrote input5.mid to test_midi
INFO:tensorflow:Wrote input6.mid to test_midi
INFO:tensorflow:Wrote input7.mid to test_midi
INFO:tensorflow:Wrote input8.mid to test_midi
INFO:tensorflow:Wrote input9.mid to test_midi
INFO:tensorflow:Wrote input10.mid to test_midi


In [18]:
for i in range(len(real_inputs)):
    real_inputs[i].pop()

In [25]:
import pickle
real_inputs_file = open("real_inputs.pkl","wb")
pickle.dump(real_inputs, real_inputs_file)
real_inputs_file.close()

In [28]:
test_file = open("real_inputs.pkl","rb")
a = []
a = pickle.load(test_file)
test_file.close()

In [34]:
index_to_char = "._0123456789abcdefghijklmnopqrstuvwxyz"
with open("Wikifonia_to_text.txt", "w") as wf:
    for i in real_inputs:
        for j in range(len(i)):
            wf.write(index_to_char[i[j]])
    wf.close()